In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('practice').getOrCreate()

In [24]:
dframe = spark.read.option('Header','true').csv('test2.csv').show

In [6]:
import pandas as pd
pd.read_csv('test2.csv')

,Name,Age,Experience,Salary
0,Edgar,31.0,12.0,30000.0
1,Flaco,21.0,3.0,50000.0
2,Rojas,54.0,4.0,40000.0
3,Amlo,56.0,23.0,60000.0
4,Adame,NaN,NaN,55000.0
5,NaN,78.0,2.0,21.0
6,NaN,9.0,NaN,NaN


In [55]:
#spark.read.option('Header','true').csv('test2.csv').show()
##import pandas as pd
##pd.read_csv('test2.csv')
dframe = spark.read.csv('test2.csv', header=True, inferSchema=True).show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Edgar|  31|        12| 30000|
|Flaco|  21|         3| 50000|
|Rojas|  54|         4| 40000|
| Amlo|  56|        23| 60000|
|Adame|null|      null| 55000|
| null|  78|         2|    21|
| null|   9|      null|  null|
+-----+----+----------+------+



In [26]:
dframe.describe().show()


+-------+-----+------------------+----------------+------------------+
|summary| Name|               Age|      Experience|            Salary|
+-------+-----+------------------+----------------+------------------+
|  count|    5|                 6|               5|                 6|
|   mean| null|              41.5|             8.8|39170.166666666664|
| stddev| null|25.633961847517835|8.87130204648675| 21996.31196738823|
|    min|Adame|                21|              12|                21|
|    max|Rojas|                 9|               4|             60000|
+-------+-----+------------------+----------------+------------------+



In [28]:
type(dframe)

pyspark.sql.dataframe.DataFrame

In [33]:
dframe.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  31|        12| 30000|
|  21|         3| 50000|
|  54|         4| 40000|
|  56|        23| 60000|
|null|      null| 55000|
|  78|         2|    21|
|   9|      null|  null|
+----+----------+------+



In [35]:
dframe.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Edgar|  31|        12| 30000|
|Flaco|  21|         3| 50000|
|Rojas|  54|         4| 40000|
| Amlo|  56|        23| 60000|
|Adame|null|      null| 55000|
| null|  78|         2|    21|
| null|   9|      null|  null|
+-----+----+----------+------+



In [41]:
dframe.na.drop().show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Edgar| 31|        12| 30000|
|Flaco| 21|         3| 50000|
|Rojas| 54|         4| 40000|
| Amlo| 56|        23| 60000|
+-----+---+----------+------+



In [42]:
dframe.na.drop(how="all").show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Edgar|  31|        12| 30000|
|Flaco|  21|         3| 50000|
|Rojas|  54|         4| 40000|
| Amlo|  56|        23| 60000|
|Adame|null|      null| 55000|
| null|  78|         2|    21|
| null|   9|      null|  null|
+-----+----+----------+------+



In [44]:
dframe.na.drop(how="any").show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Edgar| 31|        12| 30000|
|Flaco| 21|         3| 50000|
|Rojas| 54|         4| 40000|
| Amlo| 56|        23| 60000|
+-----+---+----------+------+



In [45]:
dframe.na.drop(how="any",thresh=2).show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Edgar|  31|        12| 30000|
|Flaco|  21|         3| 50000|
|Rojas|  54|         4| 40000|
| Amlo|  56|        23| 60000|
|Adame|null|      null| 55000|
| null|  78|         2|    21|
+-----+----+----------+------+



In [46]:
dframe.na.drop(how="any",subset=['Experience']).show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Edgar| 31|        12| 30000|
|Flaco| 21|         3| 50000|
|Rojas| 54|         4| 40000|
| Amlo| 56|        23| 60000|
| null| 78|         2|    21|
+-----+---+----------+------+



In [48]:
dframe.na.fill('Missing Value', 'Experience').show()

+-----+----+-------------+------+
| Name| Age|   Experience|Salary|
+-----+----+-------------+------+
|Edgar|  31|           12| 30000|
|Flaco|  21|            3| 50000|
|Rojas|  54|            4| 40000|
| Amlo|  56|           23| 60000|
|Adame|null|Missing Value| 55000|
| null|  78|            2|    21|
| null|   9|Missing Value|  null|
+-----+----+-------------+------+



In [60]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age', 'Experience', 'Salary'],
    outputCols=["{}_imputed".format(c) for c in ['Age', 'Experience', 'Salary']]
).setStrategy("mean")

In [61]:
#Add imputation cols to dframe
imputer.fit(dframe).transform(dframe).show()

+-----+----+----------+------+-----------+------------------+--------------+
| Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+------+-----------+------------------+--------------+
|Edgar|  31|        12| 30000|         31|                12|         30000|
|Flaco|  21|         3| 50000|         21|                 3|         50000|
|Rojas|  54|         4| 40000|         54|                 4|         40000|
| Amlo|  56|        23| 60000|         56|                23|         60000|
|Adame|null|      null| 55000|         41|                 8|         55000|
| null|  78|         2|    21|         78|                 2|            21|
| null|   9|      null|  null|          9|                 8|         39170|
+-----+----+----------+------+-----------+------------------+--------------+



In [59]:
dframe.dtypes

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int'), ('Salary', 'int')]

In [58]:
dframe = spark.read.csv('test2.csv', header=True, inferSchema=True)